# Demo: Serenity Scenario API

Serenity lets you run stress tests on linear products -- portfolios of digital asset tokens -- in its current version. You can run scenarios that you create yourself, called custom scenarios, or you can use canned or predefined scenarios packaged with the Serenity platform. The API lets you manage your custom scenarios, execute scenario runs and retrieve run results programmatically, so if you wish to develop your own suite of regular scenarios to run against your portfolio you can easily script them.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import datetime
import re

from datetime import date
from uuid import UUID, uuid4

import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

## Getting started

To run a scenario the first thing we need is a portfolio:

In [ ]:
from serenity_types.portfolio.core import SimplePortfolio

asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

# scenarios uses a special type, SimplePortfolio -- wrap it
simple_portfolio = SimplePortfolio(portfolio_id=uuid4(),
                                   portfolio_name='Test Portfolio',
                                   portfolio_manager='Test PM',
                                   base_currency_id=uuid4(),
                                   asset_positions=portfolio.to_asset_positions())

In [ ]:
models = api.model()

In [ ]:
model_short_name = 'risk.factor.ff.beta.mt.v1_6'  
model_meta = api.model().load_model_metadata()
model_config_id = model_meta.get_model_configuration_id(model_short_name)

The starting point is to acquire a Scenarios API wrapper:

In [ ]:
scenarios = api.scenarios()

## Listing custom scenarios

In [ ]:
custom_scenarios = scenarios.get_custom_scenarios().result
rows = []
for scenario in custom_scenarios:
    rows.append({'scenario_id': scenario.scenario_id, 'scenario_name': scenario.name})
pd.DataFrame(rows)

## Listing predefined scenarios

In [ ]:
predefined_scenarios = scenarios.get_predefined_scenarios().result
scenario_by_name = {}
rows = []
for scenario in predefined_scenarios:
    name_elements = re.split(':\s', scenario.name)
    model_config_id = UUID(name_elements[0])
    scenario_name = name_elements[1]
    rows.append({'scenario_id': scenario.scenario_id, 'model_config_id': model_config_id, 'scenario_name': scenario_name})
    scenario_by_name[scenario_name] = {'scenario': scenario, 'model_config_id': model_config_id}
pd.DataFrame(rows)

## Managing custom scenarios

### Example 1: Defining your own custom shocks

In [ ]:
from serenity_types.risk.scenarios import ScenarioCloneRequest, ScenarioDefinition, ScenarioSource

# create a custom scenario, letting the server allocated ID and version
custom = ScenarioDefinition(scenario_id=None, scenario_version=None,
                            source=ScenarioSource.CUSTOM,
                            name='Demo Custom Scenario',
                            shocks=[], last_updated_by='SDK notebook')
custom = scenarios.create_custom_scenario(custom).result

In [ ]:
# update it
custom.name='Demo Custom Scenario MODIFIED'
custom = scenarios.update_custom_scenario(custom).result

In [ ]:
# clone it
clone_request = ScenarioCloneRequest(scenario_id=custom.scenario_id, scenario_name=f'{custom.name} - CLONED')
custom_clone = scenarios.clone_scenario(clone_request).result

In [ ]:
# delete the scenario and its clone
assert scenarios.delete_custom_scenario(custom.scenario_id).result.deleted
assert scenarios.delete_custom_scenario(custom_clone.scenario_id).result.deleted

### Example 2: Using Serenity's automated Scenario generation feature

In [ ]:
from serenity_types.risk.scenarios import CustomScenarioGeneratorRequest, ScenarioDefinition

# generate a custom scenario definition by passing in the 
generator_req = CustomScenarioGeneratorRequest(
    name="Demo Auto Generated Scenario",
    start_date="2023-10-01",
    end_date="2023-10-25",
    model_config_id=model_config_id)
custom_gen = scenarios.generate_custom_scenario(generator_req).result
custom_gen

In [ ]:
from serenity_sdk.client.raw import SerenityError
from serenity_types.risk.scenarios import ScenarioRequest
from serenity_types.pricing.core import PricingContext

request = ScenarioRequest(scenario_id=custom_gen.scenario_id, portfolio=simple_portfolio,
                          model_config_id=model_config_id)

response = scenarios.run_scenario(request)
response.result

In [ ]:
# delete the auto generated scenario
assert scenarios.delete_custom_scenario(custom_gen.scenario_id).result.deleted

## Running a predefined scenario

In [ ]:
from serenity_sdk.client.raw import SerenityError
from serenity_types.risk.scenarios import ScenarioRequest
from serenity_types.pricing.core import PricingContext

scenario_info = scenario_by_name['3AC Insolvency']
predefined_scenario = scenario_info['scenario']
model_config_id = scenario_info['model_config_id']
start_date = date(2022, 12, 26)
pricing_context = PricingContext(as_of_date=start_date)
request = ScenarioRequest(scenario_id=None, scenario=predefined_scenario, portfolio=simple_portfolio,
                          pricing_context=pricing_context, model_config_id=model_config_id,
                          start_date=start_date, end_date=start_date, schema_version=1)

scenarios.run_scenario(request)